In [579]:
import tensorflow as tf
import yfinance as yf
import pandas as pd

In [580]:
from sklearn.model_selection import train_test_split


tickerName = "TSLA"
ticker = yf.Ticker(tickerName)
df = ticker.history(period = "max", interval = "1d")
df = df.drop(['Dividends', 'Stock Splits'], axis = 1)
df.to_csv(f"./stock_data/{tickerName}.csv")
df = pd.read_csv(f"./stock_data/{tickerName}.csv")

'''df = df.assign(**{'Open(t-1)': df['Open'].shift(-1)},
                **{'Close(t-1)': df['Close'].shift(-1)},
                **{'High(t-1)': df['High'].shift(-1)},
                **{'Low(t-1)': df['Low'].shift(-1)},
                **{'Volume(t-1)': df['Volume'].shift(-1)},
                **{'Open(t-2)': df['Open'].shift(-2)},
                **{'Close(t-2)': df['Close'].shift(-2)},
                **{'High(t-2)': df['High'].shift(-2)},
                **{'Low(t-2)': df['Low'].shift(-2)},
                **{'Volume(t-2)': df['Volume'].shift(-2)},
                **{'Open(t-3)': df['Open'].shift(-3)},
                **{'Close(t-3)': df['Close'].shift(-3)},
                **{'High(t-3)': df['High'].shift(-3)},
                **{'Low(t-3)': df['Low'].shift(-3)},
                **{'Volume(t-3)': df['Volume'].shift(-3)})

df = df.drop(['High', 'Low', "Volume"], axis = 1)
x = df[['Open',
        'Open(t-1)', 'Close(t-1)', 'High(t-1)', 'Low(t-1)', 'Volume(t-1)',
        'Open(t-2)', 'Close(t-2)', 'High(t-2)', 'Low(t-2)', 'Volume(t-2)',
        'Open(t-3)', 'Close(t-3)', 'High(t-3)', 'Low(t-3)', 'Volume(t-3)']].values
y = df['Close'].values
'''

df = df.dropna()
df = df.copy()

df['Date'].head()

0    2010-06-29 00:00:00-04:00
1    2010-06-30 00:00:00-04:00
2    2010-07-01 00:00:00-04:00
3    2010-07-02 00:00:00-04:00
4    2010-07-06 00:00:00-04:00
Name: Date, dtype: object

In [581]:
df.index = pd.to_datetime(df['Date'])
open = df['Open']

C:\Users\Tony\AppData\Local\Temp\ipykernel_12420\2332514939.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df.index = pd.to_datetime(df['Date'])


In [582]:
import numpy as np
def df_to_x_y(df, window_size):
    x = []
    y = []
    
    # Use multiple features (e.g., Open, Close, High, Low, Volume)
    features = ['Open', 'Close', 'High', 'Low', 'Volume']
    df_as_np = df[features].to_numpy()  # Use these features for input
    
    for i in range(len(df_as_np) - window_size):
        row = df_as_np[i:i+window_size]  # Create the sequence of multiple features
        x.append(row)
        label = df_as_np[i+window_size, 0]  # Predict the 'Open' price (or another feature)
        y.append(label)
        
    return np.array(x), np.array(y)

In [583]:
x, y = df_to_x_y(df,30)
x.shape,y.shape

((3637, 30, 5), (3637,))

In [584]:
x_train,y_train = x[:2500],y[:2500]
x_val,y_val = x[2500:3000],y[2500:3000]
x_test,y_test = x[3000:],y[3000:]

y_train = np.expand_dims(y_train, axis=-1)  # Now shape is (3637, 1)
y_val = np.expand_dims(y_val, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Normalize the data (here `x_train`, `x_val`, `x_test` should have shape (samples, features))
x_train_standard = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1]))  # Flatten for standard scaling
x_val_standard = scaler.transform(x_val.reshape(-1, x_val.shape[-1]))
x_test_standard = scaler.transform(x_test.reshape(-1, x_test.shape[-1]))

# Reshape back to 3D array
x_train_standard = x_train_standard.reshape(-1, x_train.shape[1], x_train.shape[2])
x_val_standard = x_val_standard.reshape(-1, x_val.shape[1], x_val.shape[2])
x_test_standard = x_test_standard.reshape(-1, x_test.shape[1], x_test.shape[2])

# Initialize the scaler for y
scaler_y = StandardScaler()

# Fit the scaler on y_train and transform it
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))

# Transform y_val and y_test using the fitted scaler (do not fit again)
y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))


In [585]:
import tensorflow as tf
print(tf.__version__)

2.19.0-dev20250121


In [586]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LSTM

model1 = Sequential()
model1.add(InputLayer(input_shape=(30,5)))
model1.add(LSTM(128))
model1.add(BatchNormalization())  # Batch normalization after LSTM
model1.add(Dense(32, activation="relu"))
model1.add(Dense(1, activation="linear"))
model1.summary()

c:\Users\Tony\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_48"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_52 (LSTM)                  │ (None, 128)            │        68,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_90 (Dense)                │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_91 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,281 (286.25 KB)

 Trainable params: 73,025 (285.25 KB)

 Non-trainable params: 256 (1.00 KB)

In [587]:
from tensorflow.keras.optimizers import SGD
cp = ModelCheckpoint('model1.h5', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer = SGD(learning_rate = 0.0001), metrics=[RootMeanSquaredError()])

In [588]:
from tensorflow.python.keras.engine import data_adapter

def _is_distributed_dataset(ds):
    return isinstance(ds, data_adapter.input_lib.DistributedDatasetSpec)

data_adapter._is_distributed_dataset = _is_distributed_dataset

In [589]:
model1.fit(x_train_standard, y_train_scaled, validation_data=(x_val, y_val), epochs = 100, callbacks = [cp])

Epoch 1/100
75/79 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3253 - root_mean_squared_error: 0.5651

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.3201 - root_mean_squared_error: 0.5605 - val_loss: 63873.2070 - val_root_mean_squared_error: 252.7315
Epoch 2/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0880 - root_mean_squared_error: 0.2957 - val_loss: 63968.4102 - val_root_mean_squared_error: 252.9198
Epoch 3/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0705 - root_mean_squared_error: 0.2653 - val_loss: 64101.7656 - val_root_mean_squared_error: 253.1833
Epoch 4/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0650 - root_mean_squared_error: 0.2542 - val_loss: 64286.7539 - val_root_mean_squared_error: 253.5483
Epoch 5/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0696 - root_mean_squared_error: 0.2635 - val_loss: 64534.3359 - val_root_mean_squared_error: 254.0361
Epoch 6/100
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0774 - root_mean_squared_error: 0.2765 - val_loss: 64804.3203 - val_root_mean_squared_error: 254.5669
Epoch 7/100
79/79 ━━━━━━━━━━━━━━━

In [591]:
from tensorflow.keras.models import load_model
model1 = load_model('model1.h5')
train_predictions = model1.predict(x_train).flatten()
train_results = pd.DataFrame(data = {'train_predictions':train_predictions.flatten(),'Actuals':y_train.flatten()})
train_results

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


,train_predictions,Actuals
0,-0.469395,1.246000
1,-0.469395,1.186667
2,-0.469395,1.212000
3,-0.469395,1.230000
4,-0.469395,1.264000
...,...,...
2495,-0.469395,93.066666
2496,-0.469395,110.599998
2497,-0.469395,103.733330
2498,-0.469395,102.866669
